In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import vae_cyc as vc
import pandas as pd 
import lightning as pl
import wandb
from torch.utils.data import DataLoader 
from pytorch_lightning.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [122]:
df = pd.read_csv('/workspace/uniref50_small.csv')

In [123]:
df['len'] = df.Sequence.str.len()
df = df[df.len <= 500]

In [124]:
len(df)

999999

In [125]:
vocab = vc.vocab.AminoAcidVocab()

In [116]:
# df.to_parquet('/workspace/uniref50.parquet')

In [117]:
train_df, val_df = train_test_split(df, test_size=0.001)

In [118]:
train_dataset = vc.Dataset(train_df, vocab, 'Sequence',100)

In [119]:
val_dataset = vc.Dataset(val_df, vocab, 'Sequence',100)

In [79]:
checkpoint_callback = ModelCheckpoint(dirpath='/workspace/uniref50-vae-large/', verbose=True, save_top_k=3,monitor='val/loss', every_n_train_steps=5000)

In [80]:
train_dl = DataLoader(train_dataset,batch_size=32,shuffle=True, collate_fn=train_dataset.collate,num_workers=0)
val_dl = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=val_dataset.collate, num_workers=0)

In [81]:
wandb_logger = WandbLogger(project="run_pod", log_model=True)
wandb.init()

epoch,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/kl_loss,▁▁▃▃▅▄▄▆▅▅▅▄▄▄▅▅▆▆▅▅▇▄▇▄▂▄▅▅▇▇▆█▇▇▆▆▇▅▆▆
train/kl_weight,▁▃▃▄▄▆▇█████████████████▁▁▂▃▃▄▆▆▇███████
train/loss,▅▂▅▂█▅▂▆▅▆▄▄▁▅▂▅▄▄▄▄▄▄▂▂▄▃▅▄▃▅█▄▂▆▃▄▅▆▃▃
train/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/r_loss,▄▄█▅▅▆▇▄▅▂▂▅▃▅▅█▇▂▃▆▅▃▅▅▁▁▅▃▃▅▄▇▅▇▆▄▆▅▆▂
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇███
val/kl_loss,▁▂▃▃▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▂▃▅▆▆▇▇█████▇▇▇███
val/loss,█▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▁▁▁▁▁▂▂▂▂▃▂▂▂▃▂▂▂▂
val/r_loss,█▇▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▄▅▄▂▁▁▁▁▁▂▁▁▂▁▂▁▂▂▁▁▁


In [82]:
trainer = pl.Trainer(devices=1, logger=wandb_logger, precision='16-mixed', callbacks=[checkpoint_callback])

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [84]:
model = vc.VAE(128,32,2, enc_num_layers=2, vocab=vocab, c_step=100000, n_cycles_anneal=50, lr=0.0001)

In [85]:
trainer.fit(model, train_dl, val_dl)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /workspace/uniref50-vae-large exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | encoder       | GRU              | 31.5 K | train
1 | decoder       | GRU              | 25.0 K | train
2 | mu_fc         | Linear           | 16.5 K | train
3 | logvar_fc     | Linear           | 16.5 K | train
4

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...

KeyboardInterrupt



In [109]:
o = model.multinomial_generation_batched(10, max_len=250, batch_size=10)

batch:  0
Total generated:  10
